In [1]:
import pandas as pd
import ipywidgets as widgets


In [2]:
airports = pd.read_csv("../data/aeroportsANLO.csv")
routes = pd.read_csv("../data/routesANLO.csv")

In [3]:
# creation du list from airports 
name_airports = airports['name'].tolist()
iata_airports = airports['iata'].tolist()

In [4]:
airportDepart = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)
airportDepart

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [5]:
airportArrive = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)

airportArrive

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [6]:
nb_correspondances =widgets.BoundedIntText(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Number of correpondances:',
    disabled=False
)

nb_correspondances

BoundedIntText(value=1, description='Number of correpondances:', max=5, min=1)

In [7]:
"On veut aller de {} à {} avec {} corresponances".format(airportDepart.value, airportArrive.value, nb_correspondances.value)


'On veut aller de A Coruna Airport à A Coruna Airport avec 1 corresponances'

In [8]:
# d'abord on vais ecrire une function pour trouver une route entre deux airports donnees 
def findAirportCode(airportName : str):
    '''
    fonction pour trouver le airport code ('iata') si on a une airportname 
    retour : string with 3 letters corresponding to iata of airport 
    '''
    index = name_airports.index(airportName)
    return iata_airports[index]
        

In [18]:
def connection_between_two_airports(airportArriveCode : str, airportDepartCode : str):
    '''
    fonction qui verifie si on a une liaison/route entre les deux codes 
    retour : integer qui répresent le num_IATA 
    '''
    if airportDepartCode == '' or airportArriveCode == '':
      if airportDepartCode == '' : var1 = airportArriveCode
      else : var1 = airportDepartCode
      
      tmp1 = ["airportC", "IATA"]
      tmp = dict();
      index = 0 ; 
      for row in routes.iterrows():
        if(row[1][11] == var1):
          tmp2 = [row[1][12], row[1][13]]
          tmp3 = dict(zip(tmp1, tmp2))
          tmp[index] = tmp3 
          index = index + 1
      return tmp
    else: 
      for row in routes.iterrows():
        # 11 -> arrive, 12 --> depart, 13 --> num_IATA 
        if((row[1][11] == airportArriveCode) and (row[1][12] == airportDepartCode)):
          return row[1][13]

In [10]:
# fonction qui vérifie si une les deux codes de airports ont une liaison/route 
def connection_between_two_airports_v2(airportArriveCode : str, airportDepartCode : str):
    '''
    fonction qui verifie si on a une liaison/route entre les deux codes 
    retour : integer qui répresent le num_IATA ou un dict avec {airport : IATA}
    '''
    if airportDepartCode == '' or airportArriveCode == '':
      if airportDepartCode == '' : var1 = airportArriveCode
      else : var1 = airportDepartCode
      
      tmp = dict();
      for row in routes.iterrows():
        if(row[1][11] == var1):
          tmp[row[1][12]] = row[1][13] # dict avec comme kley le code d'airport 
      return tmp
    
    else: 
      for row in routes.iterrows():
        # 11 -> arrive, 12 --> depart, 13 --> num_IATA 
        if((row[1][11] == airportArriveCode) and (row[1][12] == airportDepartCode)):
          return row[1][13]
          
#connection_between_two_airports_v2('JFK', '')


In [11]:
#connection_between_two_airports('JFK', '') # check 645 tuples 
#connection_between_two_airports('', 'JFK') # check 645 tuples 

# JFK -> LAX 
d1 = connection_between_two_airports('JFK', '') # LAX
d2 = connection_between_two_airports('', 'BOS') # LAX
#print(connection_between_two_airports('JFK', 'BOS'))

In [12]:
#compare two dict to find the same airport 
             
def compare_dict_airport_v1(dict1 : dict, dict2 : dict):
    '''
    fonction qui retourne une list avec tous les airports que les deux dictionnaires ont en commen
    retour : list des airport codes 
    '''
    possibilites_duplicates = [] 
    ########## loop d1
    for k, content_k in d1.items():
            ########### loop d2 
            for p, content_p in d2.items():
                ## test if in those dict are the same airports 
                if content_k['airportC'] == content_p['airportC']:
                    possibilites_duplicates.append(content_k['airportC'])
    return list(set(possibilites_duplicates))
  

In [13]:

#dd1 = connection_between_two_airports_v2('JFK', '') # LAX 22.2
dd1 = connection_between_two_airports_v2('BOS', '') # BOS 22.2

In [14]:

#dd1 = connection_between_two_airports('JFK', '') # LAX 24.1
dd1 = connection_between_two_airports('BOS', '') # LAX 24.1


In [16]:
#compare two dict to find the same airport 
             
def compare_dict_airport_v2(dict1 : dict, dict2 : dict):
    '''
    fonction qui retourne une list avec tous les airports que les deux dictionnaires ont en commen
    retour : list des airport codes 
    '''
    possibilites_duplicates = [] 
    ########## loop d1
    for k, content_k in d1.items():
            ########### loop d2 
            for p, content_p in d2.items():
                ## test if in those dict are the same airports 
                if content_k['airportC'] == content_p['airportC']:
                    possibilites_duplicates.append(content_k['airportC'])
    return list(set(possibilites_duplicates))
  
#compare_dict_airport_v2(dd1, dd2)


In [19]:
#TODO need to reduce the number of correspondances to continent 
%run continentFonctions.ipynb

In [17]:
def liaison_correspondances(nb_correspondances : int, airportName1 : str, airportName2 : str): 
    '''
    fonction qui en fonction de nombre de correspondances donne les codes des airports qui pouvait être une correspondance 
    retour : soit un liste avec les code des airports, soit le code d'un route (depends de nb_correspondances)
    '''
    c1 = findAirportCode(airportName1)
    c2 = findAirportCode(airportName2)

    if nb_correspondances == 0 :
        return connection_between_two_airports(c1, c2)
    elif nb_correspondances == 1 :
        d1 = connection_between_two_airports(c1, '')
        d2 = connection_between_two_airports('', c2)
        
        return compare_dict_airport_v1(d1, d2)

#liaison_correspondances(1, 'Aalborg Airport', 'Amsterdam Schiphol Airport')